In [ ]:
import pandas as pd
import json
from nltk.tokenize import sent_tokenize

import re, string
import random
import uuid

In [ ]:
topic = "AHAW VBD"

In [ ]:
features = pd.read_csv("feature_descriptors.csv",dtype=str, encoding="utf-8")

In [ ]:
training_data = pd.read_csv("dataset/Extracted data element training set.csv", dtype=str, encoding="utf-8")

In [ ]:
df_topic_data = pd.read_csv(topic + ".csv",dtype=str, encoding="utf-8")

In [ ]:
topic_features = features.loc[features['topic']==topic]

In [ ]:
feature_description_mapping = {row["feature"]:row['description'] for _, row in topic_features.iterrows()}

In [ ]:
feature_description_mapping

In [ ]:
with open("feature_description_mapping.json", "w") as fdm:
    fdm.write(json.dumps(feature_description_mapping))

In [ ]:
base_columns = ["Refid"]

In [ ]:
df_data = training_data.loc[training_data['topic']==topic][list(topic_features['feature']) + base_columns]

In [ ]:
df_data = df_data.merge(df_topic_data, on="Refid")

In [ ]:
df_data = df_data.fillna("")

In [ ]:
df_data.head()

In [ ]:
with open("small_squad_sample.json") as f:
    squad_template = json.loads(f.read())

In [ ]:
#squad_template['data'][0]['title'] = topic

In [ ]:
def unicodetoascii(text):

    TEXT = (text.
            replace('\\n', " ").
    		replace('\\xe2\\x80\\x99', "'").
            replace('\\xc3\\xa9', 'e').
            replace('\\xe2\\x80\\x90', '-').
            replace('\\xe2\\x80\\x91', '-').
            replace('\\xe2\\x80\\x92', '-').
            replace('\\xe2\\x80\\x93', '-').
            replace('\\xe2\\x80\\x94', '-').
            replace('\\xe2\\x80\\x94', '-').
            replace('\\xe2\\x80\\x98', "'").
            replace('\\xe2\\x80\\x9b', "'").
            replace('\\xe2\\x80\\x9c', '"').
            replace('\\xe2\\x80\\x9c', '"').
            replace('\\xe2\\x80\\x9d', '"').
            replace('\\xe2\\x80\\x9e', '"').
            replace('\\xe2\\x80\\x9f', '"').
            replace('\\xe2\\x80\\xa6', '...').#
            replace('\\xe2\\x80\\xb2', "'").
            replace('\\xe2\\x80\\xb3', "'").
            replace('\\xe2\\x80\\xb4', "'").
            replace('\\xe2\\x80\\xb5', "'").
            replace('\\xe2\\x80\\xb6', "'").
            replace('\\xe2\\x80\\xb7', "'").
            replace('\\xe2\\x81\\xba', "+").
            replace('\\xe2\\x81\\xbb', "-").
            replace('\\xe2\\x81\\xbc', "=").
            replace('\\xe2\\x81\\xbd', "(").
            replace('\\xe2\\x81\\xbe', ")").
            replace('\\xe2\\x80\\xa0', "").
            replace('\\x0c', "").
            replace('\\xef\\xac\\x81', "fi").
            replace('\\ufb01', "fi").
            # copyright symbol
            replace('\\xc2\\xa9', "").
            replace('\\xa9', "").
            replace('\\xe2\\x89\\xa5', "of").
            replace('\\xef\\xac\\x82', "fl").
            # degree symbol
            replace('\\xc2\\xb0', " ").
            replace('\\xb0', " ").
            replace('\\xe8', "").
            replace('\\', "")

    
                 )
    return TEXT

In [ ]:
def normalize_sentence(sentence):
    #sentence.encode('ascii').decode('utf-8', errors='ignore')
    sentence = re.sub(r'\\n', " ", sentence).strip()
    sentence = unicodetoascii(sentence)
    sentence = re.sub(r'\\\\x\w+',r'', sentence)
    #sentence = re.sub(r'[^\x00-\x7f]',r'', sentence)
    #sentence = re.sub(r'\\\\x\w{1}\d{1}', ' ', sentence)
    #sentence = re.sub(r'\\x\w{1}\d{1}', ' ', sentence)
    #sentence = re.sub(r'\\\\x\w+', ' ', sentence)
    #sentence = re.sub(r'\\\\x\d{2}', ' ', sentence)
    #sentence = re.sub(r'\\\\x\d{1}', ' ', sentence)
    #sentence = re.sub(r'\s+', ' ', sentence ).strip()
    

    return sentence

In [ ]:
def add_qas_obj(q, a, index):
    q_id = uuid.uuid4()
    qas['question'] = q
    qas['id'] = "id"
    qas['answers'] = [{
        "answer_start": index ,
        "text": a
        },{
        "answer_start": index ,
        "text": a
        },{
        "answer_start": index ,
        "text": a
        }]
    return qas

In [ ]:
squad_template = {"data":[]}

In [ ]:
def create_sentence_context(sentence, index, all_sentences):
    if len(all_sentences) > 2:
        if index == 0:
            context = [sentence] + [sentences[index+1]] + [all_sentences[index+2]]
        elif index == len(all_sentences)-1:
            context = [all_sentences[index-2]] + [all_sentences[index-1]] + [sentence]
        else:
            context = [all_sentences[index-1]] + [sentence] + [all_sentences[index+1]]
        return normalize_sentence(" ".join(context))
    return ""

In [ ]:
def create_paragraph_obj(context, qas):
    return {"context":context,"qas":qas}

The problem with code below is that it checks for the offset/index in the sentence, but when we add a new sentence, it is all out of whack
and the offset should be difference, becuase it should always be from the beginning.
Need to check index after we put together the offset.

In [ ]:
for idx, sample in df_data.iterrows():
    squad_template['data'].append({'title': sample['Bibliography'], "paragraphs":[]})
    sentences = sent_tokenize(normalize_sentence(sample['raw_text']))
    for sent_id, sentence in enumerate(sentences):
        qas = []
        context = create_sentence_context(sentence, sent_id, sentences)
        for idy, attrib in enumerate(feature_description_mapping):
            #find exact match
            if re.search(r'\b'+sample[attrib].lower()+r'\b', sentence.lower()) and len(sample[attrib])>0:
                # get index of exact match
                #start_index = re.search(r'\b'+sample[attrib].lower()+r'\b', sentence.lower()).start()
                start_index = re.search(r'\b'+sample[attrib].lower()+r'\b', context.lower()).start()
                #start_index = sentence.lower().index(sample[attrib].lower())
                q = feature_description_mapping[attrib]
                a = sample[attrib]
                qas.append(add_qas_obj(q, a, start_index))
        if len(qas)>=1:
            squad_template['data'][idx]['paragraphs'].append(create_paragraph_obj(context, qas))

In [ ]:
random.shuffle(squad_template['data'])

train_data = squad_template['data'][:int((len(squad_template['data'])+1)*.80)] #Remaining 80% to training set
test_data = squad_template['data'][int(len(squad_template['data'])*.80+1):] #Splits 20% data to test set

In [ ]:
train_data_wrapper = {"data":train_data, "version":"1.1"}
test_data_wrapper = {"data":test_data, "version":"1.1"}

In [ ]:
with open("dev-v1.1.json", "w") as d:
    d.write(json.dumps(test_data_wrapper))

In [ ]:
with open("train-v1.1.json", "w") as d:
    d.write(json.dumps(train_data_wrapper))